In [1]:
# 初始化 Jupyter Notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import re
import time
import emoji
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By

In [3]:
pd.set_option('display.max_colwidth',100)

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
account  = 你的账号
password = 你的密码

# 获取字幕和题目

In [ ]:
chapter_list = pd.read_excel("git_chapter.xlsx",sheet_name="Sheet1")
chapter_list

In [ ]:
# 定义空字典
dic = {
    "chapter_name":[],
    "sub_name"    :[],
    "sub_url"     :[],
    "caption"     :[],
    "choice"      :[],
}


# **课程资料的结构**
    # 章 chapter
    ## 节 sub
    ### 小节 sub_sub
# 上面这个结构必须想清楚


# 遍历每一章
for (chapter_name, chapter_url) in chapter_list.values:
    time.sleep(2)
    driver = webdriver.Chrome()
    driver.maximize_window()
    
    # 绕过 chrome 检测
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument",
                          {"source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""})
    driver.get(chapter_url)
    time.sleep(1)

    # 登录界面
    driver.find_element(by=By.CSS_SELECTOR,value="#account").click()
    driver.find_element(by=By.CSS_SELECTOR,value="#account").send_keys(account)
    driver.find_element(by=By.CSS_SELECTOR,value="#password").click()
    driver.find_element(by=By.CSS_SELECTOR,value="#password").send_keys(password)
    driver.find_element(by=By.CSS_SELECTOR,value="#rectTop").click()

    # 登录后还有各种点击
    time.sleep(4)
    driver.find_element(by=By.CSS_SELECTOR,value="body > div > div.loginer-body > form > button").click()
    driver.implicitly_wait(10)
    driver.find_element(by=By.CSS_SELECTOR,value="h3 > a").click()
    driver.find_element(by=By.XPATH,value="//a[@class='btn btn-lg info-bottom-btn btn-warning ng-scope']").click()
    driver.find_element(by=By.XPATH,value="//a[@class='btn btn-danger ng-binding ng-scope']").click()
    time.sleep(1)
    driver.get(chapter_url)
    time.sleep(1)



    # 进去对应的课程页面后（注意，这里一定是 `章` ），整个逻辑是这样的（后面也是对应的详细代码）：
    # 遍历此 `章` 下的 `节`
        # 看看该 `节` 有没有嵌套的 `小节` 的情况
            # 如果有嵌套 `小节` 
                # 检查当前 `节` 下面有没有 `先放视频`
                    # 如果有，就先拿这些在 `小节` 前面的视频
                    # 现在才获取 `小节` 里面的视频
            # 如果没嵌套 `小节`，那就简单多了
        # 最后一节是考试
                # 先填上去, 这样才不会使字典的键值对长度不齐
                # 考试的页面是嵌套了 html 的, 所以需要转换一下 frame
                # 拿出题目的 ABCD 选项
                # 如果有填空题
                    # 填入填空题的答案
                # 如果没有填空题
                    # 填入 "没有填空题"
                # 把选择题的信息拼接起来



    # 每一节
    subs = driver.find_elements(by=By.XPATH,value="//li[@class='ng-scope']")
    # 先进行视频学习，遍历每一节
    for sub in subs:
        if sub != subs[-1]:
            # 逐个点击
            sub.click()
            time.sleep(1)
            # 获取`节`名称
            sub_name = driver.find_element(by=By.XPATH,value="//li[@class='ng-scope']/div/div[@class='oneline ng-binding active']").text
            # 看看该 `节` 有没有嵌套的 `小节` 的情况
            sub_subs = driver.find_elements(by=By.XPATH,value="//li[@class='ng-scope']/div/div[@class='oneline ng-binding active']/following::ul/li")
            print(len(sub_subs))

            # 如果有嵌套 `小节` 
            if sub_subs != []:

                
                # ！重点来了，这是最难的地方
                
                # 遍历每一个 `小节`

                # 1. 有一些 `节` 是先有一个视频，然后再嵌套 `小节` 的  ===> try:
                # 2. 有一些 `节` 是直接嵌套 `小节` 的
                


                try:
                    # 检查当前 `节` 下面有没有 `先放视频`
                    driver.find_element(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")
                    # 如果有，就先拿这些在 `小节` 前面的视频
                    # (只有当上面这行代码没报错，下面这段 `try:` 里的代码才能被执行)
                    content = driver.find_elements(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")
                    for i in content:
                        time.sleep(1)
                        i.click()
                        driver.implicitly_wait(10)
                        time.sleep(5)
                        caption_source = driver.find_element(by=By.CSS_SELECTOR,value="div.learn-main-left > div > div > div").get_attribute("source")
                        caption = "，".join([j.split(':"')[1][:-1] for j in re.findall(r'caption":".*?"',caption_source)])
                        dic["chapter_name"].append(chapter_name)
                        dic["sub_name"    ].append(sub_name)
                        dic["sub_url"     ].append(chapter_url+"/"+sub.get_attribute("id"))
                        dic["caption"     ].append(caption)
                        dic["choice"].append("---")
                    for sub_sub in sub_subs:
                        sub_sub.click()
                        time.sleep(1)
                        sub_sub_name = driver.find_element(by=By.XPATH,value="//li[@class='ng-scope']/div/div[@class='oneline ng-binding active']").text
                        time.sleep(1)
                        # 现在才获取 `小节` 里面的视频
                        content = driver.find_elements(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")[1:]
                        # 遍历 `小节` 下的视频
                        for i in content:
                            time.sleep(1)
                            i.click()
                            driver.implicitly_wait(5)
                            time.sleep(5)
                            caption_source = driver.find_element(by=By.CSS_SELECTOR,value="div.learn-main-left > div > div > div").get_attribute("source")
                            caption = "，".join([j.split(':"')[1][:-1] for j in re.findall(r'caption":".*?"',caption_source)])
                            dic["chapter_name"].append(chapter_name)
                            dic["sub_name"    ].append(sub_name+"\n"+sub_sub_name)
                            dic["sub_url"     ].append(chapter_url+"/"+sub.get_attribute("id"))
                            dic["caption"     ].append(caption)
                            dic["choice"].append("---")
                except:
                    # 如果没有在 `小节` 前放视频，就运行下面这段
                    for sub_sub in sub_subs:
                        sub_sub.click()
                        time.sleep(1)
                        sub_sub_name = driver.find_element(by=By.XPATH,value="//li[@class='ng-scope']/div/div[@class='oneline ng-binding active']").text
                        time.sleep(1)
                        # 获取 `小节` 下的 “几个”
                        content = driver.find_elements(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")
                        # 遍历 `小节` 下的视频
                        for i in content:
                            time.sleep(1)
                            i.click()
                            driver.implicitly_wait(10)
                            time.sleep(5)
                            caption_source = driver.find_element(by=By.CSS_SELECTOR,value="div.learn-main-left > div > div > div").get_attribute("source")
                            caption = "，".join([j.split(':"')[1][:-1] for j in re.findall(r'caption":".*?"',caption_source)])
                            dic["chapter_name"].append(chapter_name)
                            dic["sub_name"    ].append(sub_name+"\n"+sub_sub_name)
                            dic["sub_url"     ].append(chapter_url+"/"+sub.get_attribute("id"))
                            dic["caption"     ].append(caption)
                            dic["choice"].append("---")

            # 如果没嵌套 `小节`，那就简单多了
            else:
                content = driver.find_elements(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")
                for i in content:
                    i.click()
                    driver.implicitly_wait(10)
                    time.sleep(5)
                    caption_source = driver.find_element(by=By.CSS_SELECTOR,value="div.learn-main-left > div > div > div").get_attribute("source")
                    caption = "，".join([j.split(':"')[1][:-1] for j in re.findall(r'caption":".*?"',caption_source)])
                    dic["chapter_name"].append(chapter_name)
                    dic["sub_name"    ].append(sub_name)
                    dic["sub_url"     ].append(chapter_url+"/"+sub.get_attribute("id"))
                    dic["caption"     ].append(caption)
                    dic["choice"].append("---")

        # 最后一节是考试
        else:
            subs[-1].click()
            time.sleep(1)
            sub_name = driver.find_element(by=By.XPATH,value="//li[@class='ng-scope']/div/div[@class='oneline ng-binding active']").text
            content = driver.find_elements(by=By.CSS_SELECTOR,value="div.resourcelist.ng-scope > div")[:1]
            for i in content:
                i.click()
                # 先填上去, 这样才不会使字典的键值对长度不齐
                dic["chapter_name"].append(chapter_name)
                dic["sub_name"    ].append(sub_name)
                dic["sub_url"     ].append(chapter_url+"/"+sub.get_attribute("id"))
                dic["caption"     ].append("---")

                time.sleep(1)
                # 考试的页面是嵌套了 html 的, 所以需要转换一下 frame
                driver.switch_to.frame(driver.find_element(by=By.CSS_SELECTOR,value="body > div.ng-scope > div > div.learn-main.clearfix > div.learn-main-left > div > div > div > iframe"))
                
                # 拿出所有题目
                quesion = [ques.text for ques in driver.find_elements(by=By.CSS_SELECTOR,value="div.queBox > div > div > div > div.ti-q > div")]
                
                # 拿出题目的 ABCD 选项
                choice_answer_A = ["A. "+ans.text for ans in driver.find_elements(by=By.CSS_SELECTOR,value="div.ti-alist.ng-scope > label:nth-child(1) > div")]
                choice_answer_B = ["B. "+ans.text for ans in driver.find_elements(by=By.CSS_SELECTOR,value="div.ti-alist.ng-scope > label:nth-child(2) > div")]
                choice_answer_C = ["C. "+ans.text for ans in driver.find_elements(by=By.CSS_SELECTOR,value="div.ti-alist.ng-scope > label:nth-child(3) > div")]
                choice_answer_D = ["D. "+ans.text for ans in driver.find_elements(by=By.CSS_SELECTOR,value="div.ti-alist.ng-scope > label:nth-child(4) > div")]
                choice_answer_true = ["正确答案: "+true.text for true in driver.find_elements(by=By.CSS_SELECTOR,value="div.answerBox > div:nth-child(2) > div")]

                # 看看有没有填空题
                fill_answer_true = [true.text for true in driver.find_elements(by=By.CSS_SELECTOR,value="div.answerBox > div:nth-child(2) > div > div")]
                
                # 如果有填空题
                if fill_answer_true != []:

                    # 填入填空题的答案
                    fill = [n for m in [[quesion[i],fill_answer_true[i-(len(quesion)-len(fill_answer_true))]] for i in range(len(quesion)-len(fill_answer_true),len(quesion))] for n in m]
                
                # 如果没有填空题
                else:

                    # 填入 "没有填空题"
                    fill = ["没有填空题"]

                # 把选择题的信息拼接起来
                choice = [n for m in [[quesion[i],choice_answer_A[i]+" "+choice_answer_B[i]+" "+choice_answer_C[i]+" "+choice_answer_D[i],choice_answer_true[i]] for i in range(len(quesion)-len(fill_answer_true))] for n in m]
                
                # 再次覆盖
                choice.extend(fill)
                dic["choice"].append("\n".join(choice))
    driver.close()
    print(chapter_name, chapter_url, emoji.emojize(":beer:",use_aliases=True)*20)
    pd.DataFrame(dic).to_excel("final.xlsx",index=False)